In [1]:
### DON'T DO "RESTART AND RUN ALL CELLS" ON THIS NOTEBOOK ###
### THERE'S A CELL THAT TAKES VERY LONG TO RUN AND IT ONLY NEEDED TO BE RUN ONCE ###

In [111]:
import pandas as pd
import numpy as np
import requests
import string
import collections
import random
import time
import beepy
import pickle
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textaugment import Wordnet
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# get dataframes from CSV files

df_occupations = pd.read_csv('../data/ESCO/occupations_en.csv')
df_occupations_aug = pd.read_csv('../data/ESCO/occupations_augmented.csv')
df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')

/var/folders/84/1j64f6yd3yd5g68_pj91ch780000gn/T/ipykernel_19953/3036819475.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')


In [4]:
# filter unneeded columns of of the dataframe and add needed ones

df_occ_n_skills = df_occ_n_skills.filter(items=['preferredLabel', 'description', 'skills'])
df_occ_n_skills = df_occ_n_skills.reindex(columns=['preferredLabel','description', 'skills'])
df_occ_n_skills.rename(columns={'preferredLabel': 'job_title'}, inplace=True)
df_occ_n_skills['description_input'] = 0
df_occ_n_skills['skills_input'] = 0

In [5]:
# create description_input and skills_input, which are strings on which the model will be fit

for row, index in tqdm(df_occ_n_skills.iterrows()):
    underscored_job_title = index['job_title'].replace(" ", "_")
    this_rows_description_input = underscored_job_title + ' ' + index['description']
    this_rows_skills_input = underscored_job_title + ' ' + index['skills']
    df_occ_n_skills.iloc[row,-2] = this_rows_description_input
    df_occ_n_skills.iloc[row,-1] = this_rows_skills_input

35824it [00:06, 5569.38it/s]


In [6]:
# inspect dataframe

df_occ_n_skills.head()

,job_title,description,skills,description_input,skills_input
0,technical director,Technical directors realise the artistic visio...,"adapt to artists' creative demands, organise r...",technical_director Technical directors realise...,technical_director adapt to artists' creative ...
1,metal drawing machine operator,Metal drawing machine operators set up and ope...,"cold drawing processes, monitor moving workpie...",metal_drawing_machine_operator Metal drawing m...,metal_drawing_machine_operator cold drawing pr...
2,precision device inspector,Precision device inspectors make sure precisio...,"precision measuring instruments, monitor machi...",precision_device_inspector Precision device in...,precision_device_inspector precision measuring...
3,air traffic safety technician,Air traffic safety technicians provide technic...,"air transport law, aircraft flight control sys...",air_traffic_safety_technician Air traffic safe...,air_traffic_safety_technician air transport la...
4,hospitality revenue manager,Hospitality revenue managers maximise revenue ...,"develop revenue generation strategies, ensure ...",hospitality_revenue_manager Hospitality revenu...,hospitality_revenue_manager develop revenue ge...


In [7]:
# get entire dataframe as dataset according to Qiewi's suggestion
# concatenate the job_title:skills list to the end of df_occ_n_skills

X_all = pd.concat([df_occ_n_skills['description_input'], df_occ_n_skills['skills_input']]).reset_index(drop=True)

In [8]:
# Preprocessing function for job descriptions

def preprocess_input(sentence):
    
    # remove punctuation
    for punctuation in string.punctuation:        
        sentence = sentence.replace(punctuation, '')
    
    # set lowercase
    sentence = sentence.lower()
    
    # remove numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(sentence)
    stopword_free_tokens = [token for token in tokens if token not in stop_words]
    sentence = ' '.join(stopword_free_tokens)

    # lemmatize
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
    
    # split into tokens again after Lemmatizing --- this was replaced by Phraser 
    # sentence = word_tokenize(sentence)
    
    return sentence

In [9]:
### run this cell & extract skills, alternative labels, and job description from the ESCO API ###
### this cell uses the original occupations dataframe (not augmented) ###

"""

# select a role
role = 'pig breeder'

# get uri for selected role
role_extract = df_occupations[df_occupations['preferredLabel'] == role]
role_uri = role_extract['conceptUri']

# get role information from the ESCO API
url = "https://ec.europa.eu/esco/api/resource/skill"
params = {
    'uri': role_uri,
    'language': 'en'
         }
api_skills = requests.get(url=url, params=params).json()

# this line gets you the whole API url; easier to read on your browser
requests.get(url=url, params=params).url

# gets a list of all skill descriptions and extracts the skill title only
api_skills_dict = api_skills.get('_links').get('hasEssentialSkill')
skills_list = [skill.get('title') for skill in api_skills_dict]

# gets list of alternative labels from the API
alt_labels_list = api_skills.get('alternativeLabel').get('en')

# gets job description from the API
job_description = api_skills.get('description').get('en').get('literal')

print('***** SKILL LIST:', skills_list)
print('***** ALTERNATIVE LABELS:', alt_labels_list)
print('***** JOB DESCRIPTION:', job_description)

"""

'\n\n# select a role\nrole = \'pig breeder\'\n\n# get uri for selected role\nrole_extract = df_occupations[df_occupations[\'preferredLabel\'] == role]\nrole_uri = role_extract[\'conceptUri\']\n\n# get role information from the ESCO API\nurl = "https://ec.europa.eu/esco/api/resource/skill"\nparams = {\n    \'uri\': role_uri,\n    \'language\': \'en\'\n         }\napi_skills = requests.get(url=url, params=params).json()\n\n# this line gets you the whole API url; easier to read on your browser\nrequests.get(url=url, params=params).url\n\n# gets a list of all skill descriptions and extracts the skill title only\napi_skills_dict = api_skills.get(\'_links\').get(\'hasEssentialSkill\')\nskills_list = [skill.get(\'title\') for skill in api_skills_dict]\n\n# gets list of alternative labels from the API\nalt_labels_list = api_skills.get(\'alternativeLabel\').get(\'en\')\n\n# gets job description from the API\njob_description = api_skills.get(\'description\').get(\'en\').get(\'literal\')\n\nprint

In [10]:
# applying preprocessing as in Gensim tutorial, also applying Phraser

def read_corpus(corpus):
   
    # instantiate Phraser outside of the loop
    sentence_stream = [entry.split(" ") for entry in corpus]
    bigrams = Phrases(
        sentence_stream,
        min_count=5,
        threshold=5,
        connector_words=ENGLISH_CONNECTOR_WORDS
        )
    
    for i, line in enumerate(corpus):    
    
        # remove punctuation
        for punctuation in string.punctuation:        
            sentence = line.replace(punctuation, '')

        # remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(sentence)
        stopword_free_tokens = [token for token in tokens if token not in stop_words]
        sentence = ' '.join(stopword_free_tokens)

        # lemmatize
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
        
        # get bigrams
        sent = sentence.split()

        # yield tagged final corpus
        yield TaggedDocument(bigrams[sent], [i])

all_corpus = list(read_corpus(X_all))

In [11]:
# initialize 'job2vec' model

job2vec_model = Doc2Vec(         # these are the parameters that got the best results so far:
    vector_size=80,              # 80
    min_count=1,                 # 1
    epochs=100,                  # 100
    window=10,                    # 5
    alpha=0.01,                  # 0.01
    dm_concat=1,                 # 1
    workers=8,                   # 8
    shrink_windows=False         # True
    )

job2vec_model.wv

In [12]:
# build vocabulary

job2vec_model.build_vocab(all_corpus)

In [13]:
### train model! ###

job2vec_model.train(
    corpus_iterable=all_corpus,
    total_examples=job2vec_model.corpus_count,
    epochs=job2vec_model.epochs
    )

In [127]:
# describe your job and get your prediction!

describe_your_job = \
"in the food and beverage industry: Waiting, Bartending, Event staff for on-site catering "




In [128]:
# preprocess your job description
new_description = preprocess_input(describe_your_job)

# detect bigrams with Gensim Phrases
sentence_stream = [entry.split(" ") for entry in X_all]
bigrams = Phrases(
    sentence_stream,
    min_count=5,
    threshold=5,
    connector_words=ENGLISH_CONNECTOR_WORDS
    )
sent = new_description.split()

# create inferred vector from your preprocessed job description
new_inferred_vector = job2vec_model.infer_vector(bigrams[sent])

# get similar job descriptions from model
similar_to_new = job2vec_model.dv.most_similar([new_inferred_vector])

In [16]:
print(f'Test Document: {new_description} \n')
print('LISTING 3 MOST SIMILAR JOB ROLES & DESCRIPTIONS \n')

for label, index in [('* MOST SIMILAR', 0), ('* 2ND MOST SIMILAR', 1), ('* 3RD MOST SIMILAR', 2)]:
    
    if similar_to_new[index][0] <= len(df_occ_n_skills):
        new_index = similar_to_new[index][0]
    else:
        new_index = similar_to_new[index][0] - len(df_occ_n_skills)

    print(label + ': ' + df_occ_n_skills.loc[new_index]['job_title'])
    print(df_occ_n_skills.loc[new_index]['description'])
    print(f'Similarity score: {round(similar_to_new[index][1]*100,1)} %')
    print(f'Index in dataframe: {new_index} \n')


Test Document: sewing machinists sew components wearing apparel together repair renovate wearing apparel hand using different sewing machines 

LISTING 3 MOST SIMILAR JOB ROLES & DESCRIPTIONS 

* MOST SIMILAR: digital artist
Digital artists create art which applies digital technology as an essential part of the creative process. Digital art is usually created using computers or more specialised digital equipment. It may be enjoyed using the same instruments, shared over the internet, or presented using more traditional media.
Similarity score: 77.0 %
Index in dataframe: 2569 

* 2ND MOST SIMILAR: industrial machine maintenance engineer
industrial machinery mechanic work on new machinery and equipment in operation. they set up for the specific application and build accoutrement if necessary, perform maintenance and repair, and run nosology to find faults in system or parts that need replacing.
Similarity score: 77.0 %
Index in dataframe: 8645 

* 3RD MOST SIMILAR: door supervisor
door s

In [17]:
beepy.beep(sound="ping")

In [21]:
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [23]:
all_corpus_embed = bert_model.encode(all_corpus)

In [129]:
new_description_embed = bert_model.encode(new_description)

In [130]:
beepy.beep(sound="ping")

In [131]:
similarity_rank = cosine_similarity([new_description_embed], all_corpus_embed)

In [132]:
similarity_rank_index = np.argsort(similarity_rank[0])[::-1]

In [112]:
# save the model to disk
filename = 'bert_model.sav'
pickle.dump(bert_model, open(filename, 'wb'))

In [135]:
# save all_corpus_embedding to disk
filename = 'all_corpus_embed.sav'
pickle.dump(all_corpus_embed, open(filename, 'wb'))

In [107]:
similarity_rank_index

array([66977, 31152, 31153, ..., 16868, 43931, 40296])

In [ ]:
# load the model from disk
filename = 'bert_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))


In [133]:
for i in range(10):
    
    print(X_all[similarity_rank_index[i]]+ '\n')

wait_staff identify customer's needs, measure customer feedback, maintain personal hygiene standards, take payments for bills, welcome restaurant guests, prepare the restaurant for service, supervise food quality, maintain relationship with customers, comply with food safety and hygiene, attend to detail regarding food and beverages, check dining room cleanliness, present menus, serve wines, alcoholic beverage products, advise guests on menus for special events, prepare hot drinks, maintain customer service, assist VIP guests, serve food in table service, serve beverages, arrange tables, dietary regimes, process payments, work in a hospitality team, prepare alcoholic beverages, assist clients with special needs, clean surfaces, take food and beverage orders from customers, assist customers, food and beverages on the menu

waiting_staff waiters/waitresses supply invitee with food and beverage as requested. waiters/waitresses usually work in restaurants, bar and hotels. this involves the